In [20]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [23]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")


<Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [24]:
df = pd.read_parquet('/home/ubuntu/data/green_tripdata_2022-01.parquet')

df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

In [26]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [27]:
df_train = read_dataframe('/home/ubuntu/data/green_tripdata_2022-01.parquet')
df_val = read_dataframe('/home/ubuntu/data/green_tripdata_2022-02.parquet')

df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [28]:

categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [29]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [30]:
train=xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

2nd way of logging, passing directly the best params to save all the params at once

In [34]:
with mlflow.start_run():

    best_params={
        'learning_rate':0.30496744373957435,
        'max_depth':86,
        'min_child_weight':19.05362965923123,
        'objective':'reg:linear',
        'reg_alpha':0.1387043209444142,
        'reg_lambda':0.26102638998865857,
        'seed':42
    }

    mlflow.log_params(best_params) #instead of calling params one by one so this at once

    booster =xgb.train(
        params = best_params,
        dtrain=train,
        num_boost_round = 10, #running for 10 iteration
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )

    y_pred=booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b","wb") as f_out:
        pickle.dump(dv, f_out)
    
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow") #logging the model, saving the framework to train the model i.e. booster int his case

[23:30:12] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:13.11946
[1]	validation-rmse:10.32095
[2]	validation-rmse:8.58064
[3]	validation-rmse:7.53932
[4]	validation-rmse:6.93805
[5]	validation-rmse:6.59704
[6]	validation-rmse:6.39979
[7]	validation-rmse:6.28321
[8]	validation-rmse:6.21490
[9]	validation-rmse:6.17382


In [35]:
import mlflow
logged_model = 'runs:/0b24065779154bccbc593a1954089442/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)


[23:33:54] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [36]:
loaded_model #its a object of type mlflow, a python function

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 0b24065779154bccbc593a1954089442

In [39]:
xgboost_model = mlflow.xgboost.load_model(logged_model) # we can do it in 2 days, above did with python model, now doing with xgboost model

[23:36:46] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [41]:
xgboost_model # this is now object of xgboost.core.Booster

In [42]:
y_pred = xgboost_model.predict(valid)
y_pred[1:10]

array([ 4.1341496, 23.627502 , 34.74984  , 23.69314  ,  8.611496 ,
       25.177982 ,  3.9820492, 15.189976 ,  5.583664 ], dtype=float32)

In [ ]:
# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(data))

Two way to log models things

    1.
    with open("models/preprocessor.b","wb") as f_out:
            pickle.dump(dv, f_out)   
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")


    2. 
    mlflow.<framework>.log_model(model, artifact_path="models/")

then load and use it
